In [ ]:
#LAND USE LAND COVER SHAPEFILES
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import os
import matplotlib.patches as mpatches

# Load the Excel file
file_path = "preprocessed_data.csv"
df = pd.read_excel(file_path, engine='openpyxl')

# Define the LULC class mapping and colors
lulc_mapping = {0: 'Waterbodies', 1: 'Cropland', 2: 'Natural Forest', 3: 'Bareland', 4: 'Grasslands'}
lulc_colors = {
    'Waterbodies': 'blue',   # Water - Blue
    'Cropland': 'yellow',    # Cropland - Yellow
    'Natural Forest': '#006400',  # Dark Green for Forest
    'Bareland': 'brown',     # Brown for Bareland
    'Grasslands': '#ADFF2F'  # Light Yellow-Green for Grasslands
}

# Extract only LULC columns
lulc_columns = ['LULC1995', 'LULC2000', 'LULC2005', 'LULC2011', 'LULC2016', 'LULC2020', 'LULC2023']

# Ensure the dataset contains latitude and longitude
required_cols = lulc_columns + ['latitude', 'longitude']
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    raise ValueError(f"Missing columns in dataset: {missing_cols}")

# Map numerical LULC values to descriptive class names
for year in lulc_columns:
    df[year] = df[year].map(lulc_mapping)

# Convert to GeoDataFrame using lat/lon for spatial data
df['geometry'] = df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")  # WGS 84 CRS

# Create output directory for the final image
output_dir = "lulc_maps/"
os.makedirs(output_dir, exist_ok=True)

# Define the number of rows and columns for the grid layout
fig, axes = plt.subplots(2, 4, figsize=(24, 14))
axes = axes.flatten()

# Generate maps for each LULC year and place in subplots
for i, year in enumerate(lulc_columns):
    ax = axes[i]

    # Shift images AND TITLES from 2016 to 2023 slightly downward
    if year in ['LULC2016', 'LULC2020', 'LULC2023']:
        shift_amount = 0.04
        ax.set_position([ax.get_position().x0, ax.get_position().y0 - shift_amount,
                         ax.get_position().width, ax.get_position().height])

    # Plot each LULC class with its corresponding color
    for lulc_class, color in lulc_colors.items():
        subset = gdf[gdf[year] == lulc_class]
        subset.plot(ax=ax, color=color, markersize=15, alpha=0.7)  # Increase marker size for clarity

    # Customize each subplot with **Larger & Bold Labels**
    title_position = 1.02 if year in ['LULC2016', 'LULC2020', 'LULC2023'] else 1.05  # Adjust title position
    ax.set_title(f"{year}", fontsize=22, color='black', fontweight='bold', pad=20, loc='center', y=title_position)

    ax.set_xlabel("Longitude", fontsize=18, color='black', fontweight='bold', labelpad=10)
    ax.set_ylabel("Latitude", fontsize=18, color='black', fontweight='bold', labelpad=10)

    # Customize axis tick labels (Latitude/Longitude values)
    ax.tick_params(axis='both', which='major', labelsize=16, labelcolor='black')

    # Make the tick labels (longitude and latitude values) **bold**
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontsize(16)
        label.set_fontweight('bold')
        label.set_color('black')

# Remove extra subplots if any
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Create a SINGLE legend (key) outside the plots
legend_patches = [mpatches.Patch(color=color, label=label) for label, color in lulc_colors.items()]
fig.legend(handles=legend_patches, title="LULC Classes", title_fontsize=18, fontsize=18, fontweight='bold'
           loc="lower center", ncol=5, frameon=False)

# Adjust layout and save as a single image
plt.tight_layout(rect=[0, 0.07, 1, 1])
final_png_path = os.path.join(output_dir, "LULC_1995_2023_combined.png")
plt.savefig(final_png_path, dpi=400, bbox_inches='tight')
plt.close()

print(f"✅ Combined LULC image with improved spacing saved at {final_png_path}")


In [ ]:
#LAND USE LAND COVER(LULC) TRENDS OVER TIME
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load dataset
df = pd.read_csv('preprocessed_data.csv')

# Map numerical LULC codes to class names
lulc_classes = {
    0: 'Waterbodies',
    1: 'Cropland',
    2: 'Natural Forest',
    3: 'Bareland',
    4: 'Grasslands'
}

# List of LULC years
years = ['LULC1995', 'LULC2000', 'LULC2005', 'LULC2011', 'LULC2016', 'LULC2020', 'LULC2023']
year_values = [1995, 2000, 2005, 2011, 2016, 2020, 2023]

# Calculate the area (count) of each LULC class for each year
lulc_counts = {}
for year in years:
    counts = df[year].value_counts().reindex(lulc_classes.keys(), fill_value=0)
    lulc_counts[year] = counts

# Convert counts to DataFrame and map class names
lulc_df = pd.DataFrame(lulc_counts)
lulc_df.index = lulc_df.index.map(lulc_classes)

# Plot all trends on a single graph
plt.figure(figsize=(12, 6))

# Trend analysis using Linear Regression
for lulc_class in lulc_df.index:
    y = lulc_df.loc[lulc_class].values.reshape(-1, 1)
    X = np.array(year_values).reshape(-1, 1)

    # Fit linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Plot actual values
    plt.plot(year_values, y.flatten(), 'o-', linewidth=2)

    # Plot trend line (dotted)
    plt.plot(year_values, model.predict(X).flatten(), '--', color='gray', linewidth=1.5)

# Customize plot with larger fonts
plt.title('LULC Class Trends Over Time (1995-2023)', fontsize=20, fontweight='bold')
plt.xlabel('Year', fontsize=18, fontweight='bold')
plt.ylabel('Area Count', fontsize=18, fontweight='bold')

# Set font size for tick labels
plt.xticks(fontsize=16, fontweight='bold')
plt.yticks(fontsize=16, fontweight='bold')

# Add a note for trend lines
plt.text(1995, max(lulc_df.max()) * 0.95, 'Dashed lines represent trends', fontsize=14, color='Black', fontweight='bold')

plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#TEMPORAL NDVI TRENDS ACROSS LULC CATEGORIES
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the reshaped dataset
file_path = "preprocessed_data.csv"
df = pd.read_csv(file_path)

# Check if 'LULC' column exists in the DataFrame
if 'LULC' not in df.columns:
    # Assuming the LULC column is named differently, for example, 'LULC2023'
    lulc_column_name = 'LULC2023'  # Replace with the correct column name
    df.rename(columns={lulc_column_name: 'LULC'}, inplace=True)

# Extract LULC category names
lulc_classes = {0: "Waterbodies", 1: "Cropland", 2: "Natural Forest", 3: "Bareland", 4: "Grasslands"}
df['LULC_Category'] = df['LULC'].map(lulc_classes)

# Extract years from column names and create a 'Year' column
years = [int(col.replace('NDVI', '')) for col in df.columns if col.startswith('NDVI')]
ndvi_df = pd.melt(df, id_vars=['LULC_Category'], value_vars=[f'NDVI{year}' for year in years],
                  var_name='Year', value_name='NDVI')
ndvi_df['Year'] = ndvi_df['Year'].str.extract('(\d+)').astype(int)

# Group by year and LULC category and calculate mean NDVI
temporal_trends = ndvi_df.groupby(['Year', 'LULC_Category'])['NDVI'].mean().reset_index()

# Plot NDVI trends over time for each LULC category
plt.figure(figsize=(12, 6))
sns.lineplot(data=temporal_trends, x='Year', y='NDVI', hue='LULC_Category', marker="o")

# Set font size and bold style for better visibility
plt.xlabel("Year", fontsize=18, fontweight='bold')
plt.ylabel("Mean NDVI", fontsize=18, fontweight='bold')
plt.title("Temporal NDVI Trends Across LULC Categories", fontsize=20, fontweight='bold')

# Set font size for tick labels
plt.xticks(fontsize=16, fontweight='bold')
plt.yticks(fontsize=16, fontweight='bold')

# Move legend to the right outside the plot with a larger font size
plt.legend(title="LULC Category", title_fontsize=18, fontsize=16, loc='upper left', bbox_to_anchor=(1, 1))

plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
#VISUALIZATION OF NDVI DISTRIBUTION ACROSS LULC CATEGORIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
file_path = "preprocessed_data.csv"
df = pd.read_csv(file_path)

# Reshape the dataset
years = [1995, 2000, 2005, 2011, 2016, 2020, 2023]
reshaped_data = []

for year in years:
    if f'NDVI{year}' in df.columns:
        temp_df = pd.DataFrame({
            'Year': year,
            'NDVI': df[f'NDVI{year}'],
            'LULC': df[f'LULC{year}'].astype(int) if f'LULC{year}' in df.columns else np.nan,
            'TMAX': df[f'TMAX{year}'] if f'TMAX{year}' in df.columns else np.nan,
            'TMIN': df[f'TMIN{year}'] if f'TMIN{year}' in df.columns else np.nan,
            'Prec': df[f'Prec{year}'] if f'Prec{year}' in df.columns else np.nan
        })
        reshaped_data.append(temp_df)

# Concatenate all years into a single dataset
df_reshaped = pd.concat(reshaped_data, ignore_index=True)

# Drop rows with missing values
df_reshaped.dropna(inplace=True)

# Feature Engineering
# Compute rolling averages for temperature and precipitation
df_reshaped['TMAX_Rolling'] = df_reshaped['TMAX'].rolling(window=3, min_periods=1).mean()
df_reshaped['TMIN_Rolling'] = df_reshaped['TMIN'].rolling(window=3, min_periods=1).mean()
df_reshaped['Prec_Rolling'] = df_reshaped['Prec'].rolling(window=3, min_periods=1).mean()

# Include lagged NDVI values
df_reshaped['NDVI_Lag1'] = df_reshaped['NDVI'].shift(1)
df_reshaped['NDVI_Lag2'] = df_reshaped['NDVI'].shift(2)
df_reshaped.dropna(inplace=True)

# Define features and target variable
X = df_reshaped[['LULC', 'TMAX', 'TMIN', 'Prec', 'TMAX_Rolling', 'TMIN_Rolling', 'Prec_Rolling', 'NDVI_Lag1', 'NDVI_Lag2']]
y = df_reshaped['NDVI']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define parameter grid for hyperparameter tuning
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform Randomized Search for faster tuning
rf_model = RandomForestRegressor(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, n_iter=10, cv=3, scoring='r2', n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# Best model from Randomized Search
best_rf_model = random_search.best_estimator_

# Predict on test data
y_pred = best_rf_model.predict(X_test)

# Evaluate model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f'Optimized RMSE: {rmse}')
print(f'Optimized R² Score: {r2}')

# Feature importance
plt.figure(figsize=(8,6))
importances = best_rf_model.feature_importances_
plt.barh(['LULC', 'TMAX', 'TMIN', 'Prec', 'TMAX_Rolling', 'TMIN_Rolling', 'Prec_Rolling', 'NDVI_Lag1', 'NDVI_Lag2'], importances)
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.title("Feature Importance in Optimized Random Forest Model")
plt.show()

# Extract LULC class-based insights
lulc_classes = {0: "Waterbodies", 1: "Cropland", 2: "Natural Forest", 3: "Bareland", 4: "Grasslands"}
df_reshaped['LULC_Category'] = df_reshaped['LULC'].map(lulc_classes)

# Compute mean NDVI per LULC category
lulc_ndvi_means = df_reshaped.groupby('LULC_Category')['NDVI'].mean().sort_values()

# Visualize NDVI distribution across LULC categories
plt.figure(figsize=(10,6))
sns.boxplot(x='LULC_Category', y='NDVI', data=df_reshaped, order=lulc_ndvi_means.index)
plt.xlabel("LULC Category")
plt.ylabel("NDVI")
plt.title("NDVI Distribution Across LULC Categories")
plt.xticks(rotation=45)
plt.show()

print("Mean NDVI per LULC Category:")
print(lulc_ndvi_means)

# Plot NDVI trends over time for each LULC category
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_reshaped.groupby(['Year', 'LULC_Category'])['NDVI'].mean().reset_index(),
             x='Year', y='NDVI', hue='LULC_Category', marker="o")
plt.xlabel("Year")
plt.ylabel("Mean NDVI")
plt.title("Temporal NDVI Trends Across LULC Categories")
plt.legend(title="LULC Category")
plt.grid(True)
plt.show()

# Extract key insights from temporal trends
ndvi_trends = df_reshaped.groupby(['Year', 'LULC_Category'])['NDVI'].mean().unstack()
print("Temporal NDVI Trends:")
print(ndvi_trends)


In [ ]:
#CONFUSION MATRIX
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Load dataset
file_path = "preprocessed_data.csv"
df = pd.read_csv(file_path)

# Define features and target variable (latest NDVI value)
target = 'NDVI2023'

# Define feature base names (without years)
feature_bases = ['LULC', 'TMAX', 'TMIN', 'Prec', 'BSI', 'SMI']

# Identify available years in the dataset
years = [1995, 2000, 2005, 2011, 2016, 2020, 2023]

# Compute averaged features
for feature in feature_bases:
    cols = [f'{feature}{year}' for year in years if f'{feature}{year}' in df.columns]
    if cols:
        df[feature + '_avg'] = df[cols].mean(axis=1)

# Define final feature set using the averaged values
features = [f'{feature}_avg' for feature in feature_bases]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)  # Predictions for Random Forest
y_pred_xgb = xgb_model.predict(X_test)  # Predictions for XGBoost

# Convert predictions to integers for confusion matrix
y_pred_rf = y_pred_rf.astype(int)
y_pred_xgb = y_pred_xgb.astype(int)
y_test = y_test.astype(int)

# Generate confusion matrices
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)

# Plot confusion matrix for Random Forest
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
sns.heatmap(conf_matrix_rf, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={"size": 14, "weight": "bold"})
plt.title("Confusion Matrix - Random Forest", fontsize=14, fontweight='bold')
plt.xlabel("Predicted", fontsize=12, fontweight='bold')
plt.ylabel("Actual", fontsize=12, fontweight='bold')

# Plot confusion matrix for XGBoost
plt.subplot(1, 2, 2)
sns.heatmap(conf_matrix_xgb, annot=True, fmt='d', cmap='Greens', cbar=False, annot_kws={"size": 14, "weight": "bold"})
plt.title("Confusion Matrix - XGBoost", fontsize=14, fontweight='bold')
plt.xlabel("Predicted", fontsize=12, fontweight='bold')
plt.ylabel("Actual", fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
#CLASSIFICATION METRICS FOR CLASS
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Replace these arrays with your actual predictions and labels from the Random Forest model
y_true = np.array([1, 2, 0, 3, 4, 2, 1, 0])
y_pred = np.array([1, 2, 0, 3, 4, 2, 0, 0])

# Ensure integer labels for consistency
y_true = y_true.astype(int)
y_pred = y_pred.astype(int)

# Generate the classification report
report = classification_report(
    y_true, y_pred,
    target_names=["Waterbodies", "Cropland", "Natural Forest", "Bareland", "Grasslands"],
    digits=2
)
print("Classification Report:\n", report)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
#RANDOM FOREST SCORE USING LAND USE LAND COVER VARIABLE ONLY
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
file_path = "preprocessed_data.csv"
df = pd.read_csv(file_path)

# Reshape the dataset
years = [1995, 2000, 2005, 2011, 2016, 2020, 2023]
reshaped_data = []

for year in years:
    temp_df = pd.DataFrame({
        'Year': year,
        'NDVI': df[f'NDVI{year}'],
        'LULC': df[f'LULC{year}'].astype(int)
    })
    reshaped_data.append(temp_df)

# Concatenate all years into a single dataset
df_reshaped = pd.concat(reshaped_data, ignore_index=True)

# Drop rows with missing values
df_reshaped.dropna(inplace=True)

# Define features and target variable
X = df_reshaped[['LULC']]
y = df_reshaped['NDVI']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train Random Forest model
rf_model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on test data
y_pred = rf_model.predict(X_test)

# Evaluate model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')


In [ ]:
#RANDOM FOREST SCORE USING LULC AND ENVIRONMENTAL VARIABLES(TEMPERATURE, PRECIPITATION, SOIL MOSITURE INDEX, BARE SOIL INDEX)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
file_path = "preprocessed_data.csv"
df = pd.read_csv(file_path)

# Reshape the dataset
years = [1995, 2000, 2005, 2011, 2016, 2020, 2023]
reshaped_data = []

for year in years:
    if f'NDVI{year}' in df.columns:
        temp_df = pd.DataFrame({
            'Year': year,
            'NDVI': df[f'NDVI{year}'],
            'LULC': df[f'LULC{year}'].astype(int) if f'LULC{year}' in df.columns else np.nan,
            'TMAX': df[f'TMAX{year}'] if f'TMAX{year}' in df.columns else np.nan,
            'TMIN': df[f'TMIN{year}'] if f'TMIN{year}' in df.columns else np.nan,
            'Prec': df[f'Prec{year}'] if f'Prec{year}' in df.columns else np.nan
        })
        reshaped_data.append(temp_df)

# Concatenate all years into a single dataset
df_reshaped = pd.concat(reshaped_data, ignore_index=True)

# Drop rows with missing values
df_reshaped.dropna(inplace=True)

# Feature Engineering
# Compute rolling averages for temperature and precipitation
df_reshaped['TMAX_Rolling'] = df_reshaped['TMAX'].rolling(window=3, min_periods=1).mean()
df_reshaped['TMIN_Rolling'] = df_reshaped['TMIN'].rolling(window=3, min_periods=1).mean()
df_reshaped['Prec_Rolling'] = df_reshaped['Prec'].rolling(window=3, min_periods=1).mean()

# Include lagged NDVI values
df_reshaped['NDVI_Lag1'] = df_reshaped['NDVI'].shift(1)
df_reshaped['NDVI_Lag2'] = df_reshaped['NDVI'].shift(2)
df_reshaped.dropna(inplace=True)

# Define features and target variable
X = df_reshaped[['LULC', 'TMAX', 'TMIN', 'Prec', 'TMAX_Rolling', 'TMIN_Rolling', 'Prec_Rolling', 'NDVI_Lag1', 'NDVI_Lag2']]
y = df_reshaped['NDVI']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define parameter grid for hyperparameter tuning
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform Randomized Search for faster tuning
rf_model = RandomForestRegressor(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, n_iter=10, cv=3, scoring='r2', n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# Best model from Randomized Search
best_rf_model = random_search.best_estimator_

# Predict on test data
y_pred = best_rf_model.predict(X_test)

# Evaluate model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f'Optimized RMSE: {rmse}')
print(f'Optimized R² Score: {r2}')


In [ ]:
# XGBOOST SCORE USING (ADVANCED FEATURES)
# Convert BSI and MSI to numeric, forcing errors to NaN (use pd.to_numeric)
df_reshaped['BSI'] = pd.to_numeric(df_reshaped['BSI'], errors='coerce')
df_reshaped['MSI'] = pd.to_numeric(df_reshaped['MSI'], errors='coerce')

# Drop rows with missing values
df_reshaped.dropna(subset=['BSI', 'MSI'], inplace=True)

# Define features and target variable
X = df_reshaped[['LULC', 'TMAX', 'TMIN', 'Prec', 'TMAX_Rolling', 'TMIN_Rolling', 'Prec_Rolling', 'NDVI_Lag1', 'NDVI_Lag2', 'BSI', 'MSI']]
y = df_reshaped['NDVI']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Manually set hyperparameters for XGBoost
xgb_params = {
    'n_estimators': 200,             # Number of trees in the model
    'max_depth': 7,                  # Maximum depth of the trees
    'learning_rate': 0.1,            # Learning rate
    'subsample': 0.8,                # Subsample ratio of the training set
    'colsample_bytree': 0.8,         # Subsample ratio of columns when building each tree
    'gamma': 0.1,                    # Minimum loss reduction required to make a further partition
    'objective': 'reg:squarederror', # Regression objective
    'random_state': 42               # For reproducibility
}

# Train XGBoost model with manually set hyperparameters
xgb_model = xgb.XGBRegressor(**xgb_params)
xgb_model.fit(X_train, y_train)

# Predict on test data
y_pred = xgb_model.predict(X_test)

# Evaluate model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f'Optimized RMSE (XGBoost): {rmse}')
print(f'Optimized R² Score (XGBoost): {r2}')

# Feature importance
plt.figure(figsize=(8,6))
importances = xgb_model.feature_importances_
plt.barh(['LULC', 'TMAX', 'TMIN', 'Prec', 'TMAX_Rolling', 'TMIN_Rolling', 'Prec_Rolling', 'NDVI_Lag1', 'NDVI_Lag2', 'BSI', 'MSI'], importances)
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.title("Feature Importance in Optimized XGBoost Model")
plt.show()


In [ ]:
#XGBoost SCORE USING BASIC FEATURES
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb

# Load your dataset
data = pd.read_csv('data_3.csv')

# Checking for non-null values in each column to ensure data is loaded correctly
print("\nChecking for non-null values in the dataset:")
print(data.notnull().sum())

# Define the features (X) and target (y)
X = data[['NDVI1995', 'NDVI2000', 'NDVI2005', 'NDVI2011', 'NDVI2016', 'NDVI2020', 'NDVI2023',
          'LULC1995', 'LULC2000', 'LULC2005', 'LULC2011', 'LULC2016', 'LULC2020', 'LULC2023',
          'TMAX1990', 'TMAX1995', 'TMAX2000', 'TMAX2005', 'TMAX2011', 'TMAX2016', 'TMAX2020', 'TMAX2021',
          'TMIN1990', 'TMIN1995', 'TMIN2000', 'TMIN2005', 'TMIN2011', 'TMIN2016', 'TMIN2020', 'TMIN2021',
          'Prec1995', 'Prec2000', 'Prec2005', 'Prec2011', 'Prec2016', 'Prec2020', 'Prec2023',
          'BSI1995', 'BSI2000', 'BSI2005', 'BSI2011', 'BSI2016', 'BSI2020', 'BSI2021',
          'SMI2016', 'SMI2020', 'SMI2021']]

y = data['NDVI2023']

# Check the shape and first few rows of X and y
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

print("First few rows of the dataset (X and y):")
print(X.head())
print(y.head())

# Ensure there are no NaN values (since we are assuming your dataset is clean)
print(f"Are there any NaN values in X? {X.isnull().sum().sum() > 0}")
print(f"Are there any NaN values in y? {y.isnull().sum() > 0}")

# Check if X and y are not empty
if X.shape[0] == 0 or y.shape[0] == 0:
    raise ValueError("No valid data remaining. Check the dataset for valid rows.")

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Shape of X_train: {X_train.shape}, Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}, Shape of y_test: {y_test.shape}")

# Initialize the XGBoost model directly
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, max_depth=7, learning_rate=0.1,
                             subsample=0.9, colsample_bytree=0.9, random_state=42)

# Train the XGBoost model
xgb_model.fit(X_train, y_train)

# Predict with the XGBoost model
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate performance
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2_xgb = r2_score(y_test, y_pred_xgb)

# Print evaluation metrics
print(f"RMSE (XGBoost): {rmse_xgb}")
print(f"R² Score (XGBoost): {r2_xgb}")


In [ ]:
#LONG SHORT-TERM MEMORY (LSTM) SCORE
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Input
from sklearn.metrics import mean_squared_error, r2_score
import random

# Define function to create and return the LSTM model with hyperparameters
def create_lstm_model(units=50, dropout_rate=0.2, lstm_layers=1, batch_size=32, epochs=50):
    model = Sequential()

    # Input layer with the shape of the data
    model.add(Input(shape=(look_back, X_seq.shape[2])))

    # Bidirectional LSTM layer
    model.add(Bidirectional(LSTM(units=units, return_sequences=True)))
    model.add(Dropout(dropout_rate))

    # Add additional LSTM layers if needed
    for _ in range(lstm_layers - 1):
        model.add(LSTM(units=units, return_sequences=True))
        model.add(Dropout(dropout_rate))

    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))

    # Dense layer
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

# Hyperparameter grid
param_dist = {
    'units': [50, 64, 128, 256],
    'dropout_rate': [0.2, 0.3, 0.5],
    'lstm_layers': [1, 2, 3],
    'batch_size': [32, 64],
    'epochs': [50, 100, 200]
}

# Random search manually
best_rmse = float('inf')
best_r2 = -float('inf')
best_model = None

for _ in range(10):  # Perform 10 random searches
    # Randomly select hyperparameters
    units = random.choice(param_dist['units'])
    dropout_rate = random.choice(param_dist['dropout_rate'])
    lstm_layers = random.choice(param_dist['lstm_layers'])
    batch_size = random.choice(param_dist['batch_size'])
    epochs = random.choice(param_dist['epochs'])

    # Create and train the model
    model = create_lstm_model(units=units, dropout_rate=dropout_rate, lstm_layers=lstm_layers, batch_size=batch_size, epochs=epochs)

    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0)

    # Predict on test data
    y_pred = model.predict(X_test)

    # Evaluate the model
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Store the best model
    if rmse < best_rmse:
        best_rmse = rmse
        best_r2 = r2
        best_model = model

print(f"Optimized RMSE (LSTM with Manual Hyperparameter Tuning): {best_rmse}")
print(f"Optimized R² Score (LSTM with Manual Hyperparameter Tuning): {best_r2}")
